# PumpItUp
## DrivenDataCompetition PumpItUp
- All data is within the same folder as notebook

In [2]:
import pandas as pd
import matplotlib as plt
import sklearn
#import seaborn as sns
import numpy as np

In [3]:
# load training-data and labels
trainingValues = pd.read_csv('4910797b-ee55-40a7-8668-10efd5c1b960.csv')
trainingLabels = pd.read_csv('0bf8bc6e-30d0-4c50-956a-603fc693d966.csv')
trainingData = pd.merge(trainingValues, trainingLabels, on= 'id')

# Investigation on data
Possible data-issues
* population == 0 -> Delete all lines with population == 0
* num_private Not in feature-description -> check what to do with it
* object-data needs to be converted in category-numbers

In [4]:
# delete population == 0
trainingData = trainingData[trainingData.population != 0]


def createCatColumnsWithCodes(data : pd.DataFrame):
    cat_columns = data.select_dtypes(['object']).columns
    if len(cat_columns) == 0:
        return data
    data[cat_columns] = data[cat_columns].astype('category')
    cat_data = data[cat_columns].apply(lambda x: x.cat.codes)
    data = data.join(cat_data, rsuffix='_cat')

    return data

#change labels to category data
trainingData = createCatColumnsWithCodes(trainingData)


In [6]:
# Show data-frame top
trainingData.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality_cat,quality_group_cat,quantity_cat,quantity_group_cat,source_cat,source_type_cat,source_class_cat,waterpoint_type_cat,waterpoint_type_group_cat,status_group_cat
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,6,2,1,1,8,6,0,1,1,0
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,6,2,2,2,5,3,1,1,1,0
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,6,2,1,1,0,1,1,2,1,0
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,6,2,0,0,3,0,0,2,1,2
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,4,4,1,1,4,2,2,2,1,0


In [31]:
trainingData[['status_group','status_group_cat']].head(20)

,status_group,status_group_cat
0,functional,0
1,functional,0
2,functional,0
3,non functional,2
5,functional,0
10,functional,0
11,functional,0
13,functional,0
15,functional,0
16,non functional,2


In [7]:
# Describe data
trainingData.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,water_quality_cat,quality_group_cat,quantity_cat,quantity_group_cat,source_cat,source_type_cat,source_class_cat,waterpoint_type_cat,waterpoint_type_group_cat,status_group_cat
count,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,...,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000,38019.000000
mean,37107.559115,447.787681,969.889634,36.074387,-6.139781,0.740788,16.076988,6.299456,281.087167,1961.399721,...,5.775875,2.274757,1.335122,1.335122,6.011494,3.877298,0.262553,2.362266,1.905047,0.829217
std,21406.803661,3706.770967,612.544787,2.586779,2.737733,15.288297,21.597996,11.303334,564.687660,263.994165,...,0.846857,0.794264,0.763660,0.763660,2.053367,2.268178,0.453973,1.762964,1.368417,0.950787
min,1.000000,0.000000,-90.000000,29.607122,-11.649440,0.000000,2.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18514.500000,0.000000,347.000000,34.715340,-8.388839,0.000000,4.000000,2.000000,40.000000,1986.000000,...,6.000000,2.000000,1.000000,1.000000,5.000000,3.000000,0.000000,1.000000,1.000000,0.000000
50%,37128.000000,0.000000,1135.000000,36.706815,-5.750877,0.000000,10.000000,3.000000,150.000000,2000.000000,...,6.000000,2.000000,1.000000,1.000000,7.000000,5.000000,0.000000,1.000000,1.000000,0.000000
75%,55505.500000,100.000000,1465.000000,37.940149,-3.597016,0.000000,16.000000,5.000000,324.000000,2008.000000,...,6.000000,2.000000,2.000000,2.000000,8.000000,6.000000,1.000000,4.000000,3.000000,2.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-1.042375,1776.000000,99.000000,67.000000,30500.000000,2013.000000,...,7.000000,5.000000,4.000000,4.000000,9.000000,6.000000,2.000000,6.000000,5.000000,2.000000


#check correlation
fig = plt.pyplot.gcf()
fig.set_size_inches(25,25)
sns.heatmap(trainingData.corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [8]:
print(trainingData.dtypes.unique())

[dtype('int64') dtype('float64')
 CategoricalDtype(categories=['2004-01-07', '2004-03-01', '2004-03-06', '2004-04-01',
                  '2004-05-01', '2004-06-01', '2004-08-01', '2004-09-01',
                  '2004-12-01', '2011-01-03',
                  ...
                  '2013-08-03', '2013-09-02', '2013-09-03', '2013-10-02',
                  '2013-10-03', '2013-11-02', '2013-11-03', '2013-12-01',
                  '2013-12-02', '2013-12-03'],
                 ordered=False)
 CategoricalDtype(categories=['0', 'A/co Germany', 'Abasia', 'Abdul', 'Abood', 'Abs',
                  'Aco/germany', 'Acra', 'Act', 'Act Mara',
                  ...
                  'Yaole', 'Yasi Naini', 'Yasini', 'Yasini Selemani',
                  'Zaburi And Neig', 'Zao', 'Zao Water Spring',
                  'Zao Water Spring X', 'Zinduka', 'Zingibali Secondary'],
                 ordered=False)
 CategoricalDtype(categories=['0', 'ABASIA', 'ABDUL', 'AC', 'ACRA', 'ACT', 'ACT MARA',
                

In [5]:
#split in labels and features
labels = trainingData['status_group_cat']
features = trainingData.drop(['status_group_cat','id'],axis=1)
no_use_columns = features.select_dtypes(['category']).columns
features = features.drop(no_use_columns, axis=1)
features_list = features.columns

#check shapes
print('labels-shape: ' + str(labels.shape))
print('features-shape: ' + str(features.shape))

labels-shape: (38019,)
features-shape: (38019, 39)


In [6]:
from sklearn.model_selection import train_test_split
#split labels and features is train-data and test-data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

#check shape
print('train_features: ', train_features.shape)
print('train_labels: ', train_labels.shape)
print('test_features: ', test_features.shape)
print('test_labels: ', test_labels.shape)

train_features:  (28514, 39)
train_labels:  (28514,)
test_features:  (9505, 39)
test_labels:  (9505,)


# Try random forest


In [7]:

from sklearn.ensemble import RandomForestClassifier


In [12]:
##fit
# create baseline
baseline_preds = 1 # this is 'functional needs repair' - so every pump needs service
baseline_errors = abs(1-test_labels)
print('Baseline_error is: ', round(np.mean(baseline_errors),2))

# train model
randomforest = RandomForestClassifier(n_estimators= 500, random_state = 42)
randomforest.fit(train_features, train_labels)

Baseline_error is:  0.93


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [13]:
#predict
predictions = randomforest.predict(test_features)


In [14]:
##verify
# create baseline
print('Baseline accuracy-score is: 0.8268 with 500 estimators. LOOKS NOT BAD :-)')
# 
print('Accuracy-Score of random-forest is: ', sklearn.metrics.accuracy_score(test_labels, predictions))

Baseline accuracy-score is: 0.8268 with 500 estimators. LOOKS NOT BAD :-)
Accuracy-Score of random-forest is:  0.8268279852709101


In [ ]:
## Try random fit
from sklearn.model_selection import RandomizedSearchCV

##create random grid
# number of trees in forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop= 1000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method for selecting samples for training each node
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}

randomforest_random = RandomizedSearchCV(estimator = randomforest, param_distributions=random_grid, n_iter = 100, cv= 3, verbose= 2, random_state= 42, n_jobs = -1)
randomforest_random.fit(train_features, train_labels)

In [ ]:
predictions_random = randomforest_random.best_estimator_.predict(test_features)
print('Accuracy-Score of random-forest is: ', sklearn.metrics.accuracy_score(test_labels, predictions_random))

In [ ]:
best_estimator = randomforest_random.best_estimator_
print(randomforest_random.best_params_)

In [ ]:
# Use parameters from randomizedSearchCV
randomforest = RandomForestClassifier(n_estimators= 500, random_state = 42, min_samples_split = 10, min_samples_leaf = 2, max_depth = 100, bootstrap = False)
randomforest.fit(features, labels)
predictions = randomforest.predict(test_features)
print ('Accuracy-Score of randomforest is: ', sklearn.metrics.accuracy_score(test_labels, predictions))

In [ ]:
##Create output for competition
# prepare competition features
competition_features = pd.read_csv('702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
competition_features_prepared = createCatColumnsWithCodes(competition_features)
competition_features_prepared = competition_features_prepared.drop(['id'],axis=1)
no_use_columns = competition_features_prepared.select_dtypes(['category']).columns
competition_features_prepared = competition_features_prepared.drop(no_use_columns, axis=1)

predictions = randomforest.predict(competition_features_prepared)
competition_result = pd.read_csv('702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv')
competition_result['status_group'] = predictions
competition_result = competition_result[['id', 'status_group']]
competition_result['status_group'] = competition_result['status_group'].replace([0,1,2],['functional','functional needs repair', 'non functional'])


In [ ]:
#create output csv
competition_result.to_csv('predictions2.csv', index=False)

## Try SVC

In [17]:
from sklearn.svm import SVC

In [18]:
classifier = SVC(gamma='scale')

In [19]:
#fit
classifier.fit(train_features, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [20]:
predictions = classifier.predict(test_features)

In [21]:
sklearn.metrics.accuracy_score(test_labels, predictions)

0.5570752235665439